<a href="https://colab.research.google.com/github/fahad110011/civic-price-predictor/blob/main/notebooks/03_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install xgboost scikit-learn


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb


In [3]:
# Load the Civic subset from GitHub
url = (
  "https://raw.githubusercontent.com/"
  "fahad110011/civic-price-predictor/"
  "main/data/raw/civic_raw.csv"
)
df = pd.read_csv(url)

# Quick sanity check
df.shape, df.head()


((5784, 6),
      year  odometer  condition transmission state  price
 0  2020.0    9954.0       good        other    al  26590
 1  2018.0   28942.0       good        other    al  18590
 2  2018.0    7885.0       good        other    al  21590
 3  1995.0  215000.0       good       manual    al   3500
 4  2003.0  236890.0  excellent    automatic    al   3950)

In [4]:
# Features and target
X = df[['year', 'odometer']]
y = df['price']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [6]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Predict
y_pred = model.predict(X_test)

# Compute MSE then RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Baseline RMSE: ${rmse:,.0f}")


Baseline RMSE: $4,964
